In [ ]:
# pip install xlsxwriter

In [ ]:
import gensim
from gensim import corpora
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import os
import re
import csv
# import xlsxwriter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
df = pd.read_csv('./drive/MyDrive/sample_kw_distrib_table.tsv', sep = '\t', 
                 index_col = 0, header = [0,1,2])

CPU times: user 2.07 s, sys: 119 ms, total: 2.19 s
Wall time: 2.99 s


In [ ]:
lst = []
for line in df.columns:
  sub = [line[0], line[1], line[2]]
  lst.append(sub)
print(len(lst))
print(lst[1])

7280
['я_PRON', 'ваш', 'DET']


In [ ]:
raw_list =[]

for index, rows in df.iterrows():
    # Create list for the current row
    my_list =[rows.я_PRON]
    print(my_list)
      
    # append the list to the final list
    raw_list.append(my_list)
print(len(raw_list))

In [ ]:
foo = './drive/MyDrive/fpi_1045/'
#foo = './drive/MyDrive/rscf/rusidiolect_data/processed_spacy/fpi_1045/'
fns = os.listdir(foo)
fns1 = [x[:-4].split('_') for x in fns]
len(fns)

In [ ]:
final = []  # список для лемм типа lemma_POS
file_names = []  # список для названий файлов, чтобы удобнее было составить таблицу
kw_freq = []  # частота КС
c = 1
for name in sorted(fns):
  fn=foo + '/' + name  # путь к файлу
  with open(fn, newline='') as file:
    file_reader = csv.reader(file, delimiter=",")  # считываем файл
    lst = []  # подсписок для лемм определённого текста, который позже мы добавим в final
    for line in file_reader:
        lst.append(line[2]+'_'+line[3])  # соединяем лемму и часть речи
    final.append(lst)
    file_names.append(name)
    count_fr = []
    for kw in ['я_PRON']:      
      kw_freq.append(lst.count(kw))
  c += 1
  if c % 100 == 0:
   print(c) 

IndexError: ignored

In [ ]:
print(kw_freq)

In [ ]:
len(raw_list[0][0])

4994

In [ ]:
r = raw_list[0][0]

In [ ]:
r.sort_values(ascending=False)[:25]

In [ ]:
name_word_freq = []
t = 0
for raw in raw_list:  
  x = str(raw)
  f_name = re.findall('Name\:.+\, L', x)
  f_name = f_name[0].replace('Name: ', '')
  f_name = f_name.replace(', L', '')
  word_pos = raw[0].to_dict()
  freq = raw[0].tolist()
  for y, z in zip(word_pos, freq):
    to_add = [f_name, y[0]+'_'+y[1], z]
    t += 1
    #if t%100000 == 0:
    #  print(t)
    name_word_freq.append(to_add)

In [ ]:
# model=Word2Vec.load('./drive/MyDrive/ruscorpora_1_300_10.bin')
# model=Word2Vec.load('./drive/MyDrive/test_w2v_corpus.model')
# model=Word2Vec.load('./drive/MyDrive/web_0_300_20.bin')

In [ ]:
import gensim.models.keyedvectors as word2vec

model=word2vec.KeyedVectors.load_word2vec_format('./drive/MyDrive/model.bin', 
                                                 binary=True, unicode_errors='ignore')

In [ ]:
kw_vectors = []
for i in sorted(name_word_freq):
  if int(i[2]) > 0:
    try:
      vector = model.wv[i[1]] * int(i[2])
      print(i)
      lst = [i[0], vector]
      kw_vectors.append(lst)
    except:
      pass

In [ ]:
len(kw_vectors)

19887

In [ ]:
kw_vectors[0]

In [ ]:
final_vect = []
vect = []
names_to_check = []
check = kw_one[0]

for kw_one in kw_vectors:

  # print(len(kw_one))

  length = 0
  # print(check)
  if kw_one[0] == check:
    vect.append(kw_one[1])
    if check not in names_to_check:
      names_to_check.append(check)
  else:
    aver = sum(vect)/len(vect)
    to_add = [check, aver]
    final_vect.append(to_add)
    check = kw_one[0]
    vect = []
    vect.append(kw_one[1])
    if check not in names_to_check:
      names_to_check.append(check)
  length += 1

  if length == len(final_vect):
    aver = sum(vect)/len(vect)
    to_add = [check, aver]
    final_vect.append(to_add)
    length = 0    
    if check not in names_to_check:
      names_to_check.append(check)

#for name in file_names:
#  if name.replace('.csv', '') not in names_to_check:
#    to_add = [name.replace('.csv', ''), np.zeros(300,)]
#    final_vect.append(to_add)

print(len(final_vect))

In [ ]:
rows = []  # подсписки с названиями файлов и частотами ключевых слов и контекстных векторых измерений
for x,in final_vect:
  one = []
  one.append(x[0])
  for i in x[1]:
    one.append(i)
  rows.append(one)

print(len(rows))

In [ ]:
first_row = [' ']  # первым рядом будет заголовок ключевого слова (сначала название текста, потом слово)
second_row = [' ']  # второй ряд посвящён векторным измерениям

id_freq = 0
start_1 = 0
end_1 = 299
for test in ['я_PRON']:
    first_row.append(test)
    first_row.append(' ')
    for i in range(start_1, end_1):
      first_row.append(' ')
    second_row.append('count')
    dim = 1
    for i in range(start_1, end_1):
      second_row.append(dim)
      dim += 1

In [ ]:
print(len(rows[0]))
print(len(first_row))

302
302


In [ ]:
new_list = [first_row, second_row]
for r in rows:
  new_list.append(r)

with xlsxwriter.Workbook('./drive/MyDrive/ruscorpora_1_300_10_test.xlsx') as workbook:
    worksheet = workbook.add_worksheet()

    for row_num, data in enumerate(new_list):
        worksheet.write_row(row_num, 0, data)

In [ ]:
%%time
df_1 = pd.read_excel('./drive/MyDrive/ruscorpora_1_300_10_test.xlsx', header=[0, 1], index_col=0)

CPU times: user 2.36 s, sys: 8.69 ms, total: 2.37 s
Wall time: 2.37 s


In [ ]:
df_1

In [ ]:
import glob

w = glob.glob('./drive/MyDrive/rusidiolect_data/processed_spacy/lex_contexts/big5_536/*.tsv')  # менять путь

for way in w:
  df = pd.read_csv(way, sep = '\t', index_col = 0, header = [0])
  lst = []
  way = way.replace('./drive/MyDrive/rusidiolect_data/processed_spacy/lex_contexts/big5_536/', '')
  way = way.replace('.tsv', '')
  for line in df.columns:
    try:
      line = line.split('_')
      sub = [way, line[0], line[1]]
      lst.append(sub)
    except:
      sub = [way, 'count', ' ']
      lst.append(sub)

  raw_list =[]

  for index, rows in df.iterrows():
      # Create list for the current row
      my_list =[]
      my_list.append(index)
      my_list.append(rows)
      
      # append the list to the final list
      raw_list.append(my_list)
  # print(len(raw_list))

  filenames = []
  name_word_freq = []
  t = 0
  for raw in raw_list:  
    x = str(raw)
    f_name = raw[0]
    word_pos = raw[1].to_dict()
    freq = raw[1].tolist()
    for y, z in zip(word_pos.keys(), freq):
      to_add = [f_name, y, z]
      t += 1
      #if t%100000 == 0:
      #  print(t)
      name_word_freq.append(to_add)
    filenames.append(f_name)

  import gensim.models.keyedvectors as word2vec

  model=word2vec.KeyedVectors.load_word2vec_format('./drive/MyDrive/model.bin', binary=True, unicode_errors='ignore')

  kw_vectors = []
  for i in sorted(name_word_freq):
    if int(i[2]) > 0:
      if i[1] in model.vocab:
        #vector = model.wv[i[1]] * int(i[2])
        c = 0
        while c < int(i[2]):
          vector = model.wv[i[1]]
          lst = [i[0], vector]
          kw_vectors.append(lst)
          c += 1
        #if way == 'я_PRON':
        #    if i[0] == '10_23_ж_0_повествование':
        #      print(i[1], i[2], model.wv[i[1]])

  final_vect = []
  vect = []
  names_to_check = []
  check = kw_vectors[0][0]

  length = 0

  for kw_one in kw_vectors:

    # print(len(kw_one))

    # print(check)
    if check is kw_one[0]:
      vect.append(kw_one[1])
      if check not in names_to_check:
        names_to_check.append(check)
    else:
      aver = sum(vect)/len(vect)
      to_add = [check, aver]
      # print(to_add[0])  # имя файла
      final_vect.append(to_add)
      check = kw_one[0]
      vect = []
      vect.append(kw_one[1])
      if check not in names_to_check:
        names_to_check.append(check)
    length += 1

    if length == len(kw_vectors):  # менять число (кол-во строк-текстов)
      aver = sum(vect)/len(vect)
      to_add = [check, aver]
      # print(to_add[0])
      final_vect.append(to_add)
      length = 0    
      if check not in names_to_check:
        names_to_check.append(check)

  for name in sorted(filenames):
    if name not in names_to_check:
      to_add = [name, np.zeros(300,)]   
      final_vect.append(to_add)

  print(len(final_vect))

  rows = []  # подсписки с названиями файлов и частотами ключевых слов и контекстных векторых измерений
  for x in final_vect:
    one = []
    one.append(x[0])
    for i in x[1]:
      one.append(i)
    rows.append(one)

  #print(len(rows))

  first_row = [' ']  # первым рядом будет заголовок ключевого слова (сначала название текста, потом слово)
  second_row = [' ']  # второй ряд посвящён векторным измерениям

  id_freq = 0
  start_1 = 0
  end_1 = 300
  for test in [way]:
      first_row.append(test)
      for i in range(start_1, end_1):
        first_row.append(' ')
      dim = 1
      for i in range(start_1, end_1):
        second_row.append(dim)
        dim += 1

  #print(len(rows[0]))
  #print(len(first_row))

  new_list = [first_row, second_row]
  for r in rows:
    new_list.append(r)

  df_1 = pd.DataFrame(data=new_list)
  df_1.to_csv('./drive/MyDrive/rusidiolect_data/processed_spacy/w2v_contexts/big5_536/'+way+'.tsv', sep='\t', index=False, header=False)

  %%time
  # df_1 = pd.read_excel('./drive/MyDrive/w2v_contexts/big5_536/'+way+'.tsv', header=[0, 1], index_col=0)

  print(way+' is ready!')

In [ ]:
model['деньга_NOUN']  # ЭТО ПОСЛЕДНЯЯ ВЕРСИЯ, ВАНЯ!!! 10.11.21